In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import torch
model = torch.hub.load('pytorch/vision:v0.9.0', 'resnet101', pretrained=True)
# or any of these variants
# model = torch.hub.load('pytorch/vision:v0.9.0', 'resnet34', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.9.0', 'resnet50', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.9.0', 'resnet101', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.9.0', 'resnet152', pretrained=True)
model.eval()

Downloading: "https://github.com/pytorch/vision/archive/v0.9.0.zip" to /root/.cache/torch/hub/v0.9.0.zip
Downloading: "https://download.pytorch.org/models/resnet101-5d3b4d8f.pth" to /root/.cache/torch/hub/checkpoints/resnet101-5d3b4d8f.pth


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
# Download ImageNet labels
!wget https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt

--2021-05-03 09:05:55--  https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10472 (10K) [text/plain]
Saving to: ‘imagenet_classes.txt’

imagenet_classes.tx 100%[===================>]  10.23K  --.-KB/s    in 0s      

2021-05-03 09:05:55 (97.8 MB/s) - ‘imagenet_classes.txt’ saved [10472/10472]



In [ ]:
data_dir = '/content/drive/MyDrive/Dog (1)'

All pre-trained models expect input images normalized in the same way,
i.e. mini-batches of 3-channel RGB images of shape `(3 x H x W)`, where `H` and `W` are expected to be at least `224`.
The images have to be loaded in to a range of `[0, 1]` and then normalized using `mean = [0.485, 0.456, 0.406]`
and `std = [0.229, 0.224, 0.225]`.

Here's a sample execution.

In [ ]:
import os
from PIL import Image
from torchvision import transforms

In [ ]:
for i in os.listdir(data_dir):
  path = os.path.join(data_dir,i)
  input_image = Image.open(path)
  preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),])
  input_tensor = preprocess(input_image)
  input_batch = input_tensor.unsqueeze(0)
  with open("imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]
  
  if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model.to('cuda')

  with torch.no_grad():
    output = model(input_batch)
    prob = torch.nn.functional.softmax(output[0], dim=0)
    top5_prob, top5_catid = torch.topk(prob, 5)
    print(path)
    for i in range(top5_prob.size(0)):
      print(categories[top5_catid[i]], top5_prob[i].item())

/content/drive/MyDrive/Dog (1)/cat.jpg
Angora 0.3244708478450775
Persian cat 0.06962599605321884
Samoyed 0.04014185816049576
lynx 0.02591121755540371
West Highland white terrier 0.022636543959379196
/content/drive/MyDrive/Dog (1)/hus.jfif
Siberian husky 0.4655456840991974
Eskimo dog 0.4259459674358368
dingo 0.018962489441037178
timber wolf 0.015034133568406105
white wolf 0.014786927029490471
/content/drive/MyDrive/Dog (1)/ger.jfif
German shepherd 0.9835678339004517
malinois 0.006458834279328585
bloodhound 0.0028737152460962534
Brabancon griffon 0.0015031087677925825
muzzle 0.0010776033159345388
/content/drive/MyDrive/Dog (1)/bag.jfif
beagle 0.21758995950222015
Cardigan 0.1109733060002327
golden retriever 0.07584520429372787
Brittany spaniel 0.07314099371433258
Sussex spaniel 0.07099752128124237
/content/drive/MyDrive/Dog (1)/COR.jfif
Pembroke 0.7477830648422241
Cardigan 0.16914674639701843
German shepherd 0.05136843025684357
collie 0.005887647625058889
red fox 0.00414235470816493
/cont